In [ ]:
# Exercise 1:
#Extend the script to calculate: 
#  Broadcast address
# First and last usable host addresses
# Number of usable hosts
# Change the IP address to have it CIDR prefix 
# ( e.g. /24) and compare resulting networks

In [ ]:
import ipaddress
import socket

def analyse_ip(ip_str):
    # Create an IP interface object
    ip = ipaddress.ip_interface(ip_str)
    network = ip.network
    
    print(f"Address: {ip.ip}")
    print(f"Network: {network}")
    print(f"Netmask: {network.netmask}")
    print(f"Broadcast Address: {network.broadcast_address}")
    print(f"First Usable Host: {list(network.hosts())[0] if network.num_addresses > 2 else 'N/A'}")
    print(f"Last Usable Host: {list(network.hosts())[-1] if network.num_addresses > 2 else 'N/A'}")
    print(f"Number of Usable Hosts: {network.num_addresses - 2 if network.num_addresses > 2 else 0}")
    print(f"Is Private: {ip.ip.is_private}")
    print(f"Is Global: {ip.ip.is_global}")
    
    # List all hosts in the network if it's small
    if network.num_addresses < 256:
        print("\nHosts in network:")
        for host in network.hosts():
            print(host)

def get_device_ip():
    hostname = socket.gethostname()
    ip_address = socket.gethostbyname(hostname)
    print(f"Your Computer Name: {hostname}")
    print(f"Your Computer IP Address: {ip_address}")
    return ip_address

# Example usage
analyse_ip('192.168.1.1/24')
get_device_ip()


In [ ]:
# Exercise 2: Analyse your IP address to know if it is private/public and the
# network details.


In [ ]:
import ipaddress
import socket

def analyse_ip(ip_str):
    # Create an IP interface object
    ip = ipaddress.ip_interface(ip_str)
    network = ip.network
    
    print(f"Address: {ip.ip}")
    print(f"Network: {network}")
    print(f"Netmask: {network.netmask}")
    print(f"Broadcast Address: {network.broadcast_address}")
    print(f"First Usable Host: {list(network.hosts())[0] if network.num_addresses > 2 else 'N/A'}")
    print(f"Last Usable Host: {list(network.hosts())[-1] if network.num_addresses > 2 else 'N/A'}")
    print(f"Number of Usable Hosts: {network.num_addresses - 2 if network.num_addresses > 2 else 0}")
    print(f"Is Private: {ip.ip.is_private}")
    print(f"Is Global: {ip.ip.is_global}")
    
    # List all hosts in the network if it's small
    if network.num_addresses < 256:
        print("\nHosts in network:")
        for host in network.hosts():
            print(host)

def get_device_ip():
    hostname = socket.gethostname()
    ip_address = socket.gethostbyname(hostname)
    print(f"Your Computer Name: {hostname}")
    print(f"Your Computer IP Address: {ip_address}")
    
    # Analyse the device's IP address
    analyse_ip(f"{ip_address}/24")
    
    return ip_address

# Example usage
get_device_ip()


In [ ]:
# Exercise 3: Get the university website IP address and analyse it. Hint you
# can use seminar of week2 to know how to get the address.

In [ ]:
import ipaddress
import socket

def analyse_ip(ip_str):
    # Create an IP interface object
    ip = ipaddress.ip_interface(ip_str)
    network = ip.network
    
    print(f"Address: {ip.ip}")
    print(f"Network: {network}")
    print(f"Netmask: {network.netmask}")
    print(f"Broadcast Address: {network.broadcast_address}")
    print(f"First Usable Host: {list(network.hosts())[0] if network.num_addresses > 2 else 'N/A'}")
    print(f"Last Usable Host: {list(network.hosts())[-1] if network.num_addresses > 2 else 'N/A'}")
    print(f"Number of Usable Hosts: {network.num_addresses - 2 if network.num_addresses > 2 else 0}")
    print(f"Is Private: {ip.ip.is_private}")
    print(f"Is Global: {ip.ip.is_global}")
    
    # List all hosts in the network if it's small
    if network.num_addresses < 256:
        print("\nHosts in network:")
        for host in network.hosts():
            print(host)

def get_device_ip():
    hostname = socket.gethostname()
    ip_address = socket.gethostbyname(hostname)
    print(f"Your Computer Name: {hostname}")
    print(f"Your Computer IP Address: {ip_address}")
    
    # Analyse the device's IP address
    analyse_ip(f"{ip_address}/24")
    
    return ip_address

def get_university_ip(university_url):
    try:
        university_ip = socket.gethostbyname(university_url)
        print(f"\nUniversity Website: {university_url}")
        print(f"University IP Address: {university_ip}")
        
        # Analyse the university website's IP address
        analyse_ip(f"{university_ip}/24")
    except socket.gaierror:
        print("Could not resolve university website IP.")

# Example usage
get_device_ip()
get_university_ip("gold.ac.uk")


In [ ]:
# Exercise 5: use the client sever example from 
# previous week to have a TCP server as DHCP server
#  and create a client that would get the IP address
#  from the server following the process above.

In [ ]:
import socket
import time

# Server Configuration (IP Pool)
server = {
    "ip_pool": ["192.168.1.100", "192.168.1.101", "192.168.1.102"],
    "leases": {}
}

# Server listening for a client connection
def start_dhcp_server():
    host = '127.0.0.1'  # Localhost
    port = 12345         # Port to bind to
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind((host, port))
    server_socket.listen(1)
    
    print("[SERVER] Waiting for client to connect...")

    while True:
        conn, addr = server_socket.accept()
        print(f"[SERVER] Connection established with {addr}")
        
        try:
            # Step 2: Receive DHCPDISCOVER
            data = conn.recv(1024).decode()
            if data == "DHCPDISCOVER":
                print("[SERVER] Received DHCPDISCOVER")
                
                # Step 3: Send DHCPOFFER
                if not server["ip_pool"]:
                    print("[SERVER] No IPs available")
                    conn.send("NO_IP_AVAILABLE".encode())
                    continue
                
                offered_ip = server["ip_pool"].pop(0)
                offer_message = f"DHCPOFFER {offered_ip}"
                conn.send(offer_message.encode())
                print(f"[SERVER] Sent {offer_message}")
                
                # Step 4: Receive DHCPREQUEST
                data = conn.recv(1024).decode()
                if data.startswith("DHCPREQUEST"):
                    requested_ip = data.split()[1]
                    print(f"[SERVER] Received DHCPREQUEST for {requested_ip}")
                    
                    # Step 5: Send DHCPACK
                    server["leases"][addr] = requested_ip
                    ack_message = f"DHCPACK {requested_ip}"
                    conn.send(ack_message.encode())
                    print(f"[SERVER] Sent {ack_message}")
        except Exception as e:
            print(f"[SERVER] Error: {e}")
        finally:
            conn.close()
            print("[SERVER] Connection closed.")

# Start the server
if __name__ == "__main__":
    start_dhcp_server()


In [ ]:
import socket

# Client Configuration
client = {
    "mac": "AA:BB:CC:DD:EE:FF",
    "ip": None
}

# Client sending DHCPDISCOVER and receiving DHCPOFFER
def start_dhcp_client():
    host = '127.0.0.1'  # Localhost
    port = 12345         # Port to connect to
    
    client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client_socket.connect((host, port))
    
    try:
        # Step 1: Send DHCPDISCOVER
        discover_message = "DHCPDISCOVER"
        client_socket.send(discover_message.encode())
        print("[CLIENT] Sent DHCPDISCOVER")
        
        # Step 2: Receive DHCPOFFER
        data = client_socket.recv(1024).decode()
        print(f"[CLIENT] Received: {data}")
        
        if data.startswith("DHCPOFFER"):
            offered_ip = data.split()[1]
            print(f"[CLIENT] Offered IP: {offered_ip}")
            
            # Step 3: Send DHCPREQUEST
            request_message = f"DHCPREQUEST {offered_ip}"
            client_socket.send(request_message.encode())
            print(f"[CLIENT] Sent {request_message}")
            
            # Step 4: Receive DHCPACK
            data = client_socket.recv(1024).decode()
            print(f"[CLIENT] Received: {data}")
            
            if data.startswith("DHCPACK"):
                client["ip"] = data.split()[1]
                print(f"[CLIENT] IP address assigned: {client['ip']}")
                
    except Exception as e:
        print(f"[CLIENT] Error: {e}")
    finally:
        client_socket.close()

# Start the client
if __name__ == "__main__":
    start_dhcp_client()
